In [21]:
from scipy.interpolate.interpnd import LinearNDInterpolator, NDInterpolatorBase, \
     CloughTocher2DInterpolator, _ndim_coords_from_arrays
from scipy.spatial import cKDTree
from scipy.interpolate import RegularGridInterpolator
import numpy as np 

def griddata_custom(points, values, xi, method='linear', fill_value=np.nan,
             rescale=False):
    """
    Taken from scipy.interpolate._ndgriddata.py
    """
    points = _ndim_coords_from_arrays(points)
    if points.ndim < 2:
        ndim = points.ndim
    else:
        ndim = points.shape[-1]
    if ndim == 1 and method in ('nearest', 'linear', 'cubic'):
        from scipy.interpolate._interpolate import interp1d
        points = points.ravel()
        if isinstance(xi, tuple):
            if len(xi) != 1:
                raise ValueError("invalid number of dimensions in xi")
            xi, = xi
        # Sort points/values together, necessary as input for interp1d
        idx = np.argsort(points)
        points = points[idx]
        values = values[idx]
        if method == 'nearest':
            fill_value = 'extrapolate'
        ip = interp1d(points, values, kind=method, axis=0, bounds_error=False,
                      fill_value=fill_value)
        return ip(xi)
    elif method == 'nearest':
        ip = RegularGridInterpolator(points, values)
        return ip(xi)
    elif method == 'linear':
        ip = Akima2DInterpolator(points, values)
        return ip(xi)
    elif method == 'cubic' and ndim == 2:
        ip = CloughTocher2DInterpolator(points, values, fill_value=fill_value,
                                        rescale=rescale)
        return ip(xi)
    else:
        raise ValueError("Unknown interpolation method %r for "
                         "%d dimensional data" % (method, ndim))

In [23]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
 # !!! this actually generates a surface

df = np.load(r"reconstructed_point_cloudBSESI.npy")

keep = np.where(df[:,2] <= 0)

df = df[keep]

x = df[:,0]
y = df[:,1]
z = df[:,2]

xi = np.linspace(x.min(), x.max(), 100)
yi = np.linspace(y.min(), y.max(), 100)

X,Y = np.meshgrid(xi,yi)

Z = griddata_custom((x,y),z,(X,Y), method='cubic')

griddata

fig = go.Figure(go.Surface(x=xi,y=yi,z=Z))
fig.write_html('real_surface3.html')